In [ ]:
### Uncomment the following lines to install the required packages###
# # For Microsoft Entra ID token authentication support 
# ! pip install azure-identity 
# # For Azure OpenAI Assistant 
# ! pip install "autogen-ext[openai]"
# # This module provides the main entry point for the autogen_agentchat package 
# ! pip install autogen-agentchat
# # For Azure AI Projects client library from Azure AI Foundry SDK packages and services
# ! pip install azure-ai-projects

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.ai.projects.models import CodeInterpreterTool
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import BingGroundingTool
from autogen_agentchat.ui import Console

In [ ]:
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

In [ ]:
az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment="<your deployment model name>",  # e.g., "gpt-4o"
    api_version="<model version>",  # e.g., "2023-05-15"
    model = "<your deployment model name>",  # e.g., "gpt-4o"
    azure_endpoint="<Azure OpenAI Service endpoint>",
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key="<Your API Key from AI foundry>", # For key-based authentication.
)

In [ ]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str='<Your Project connection string from Azure AI Foundry>',
)

In [ ]:
bing_connection = project_client.connections.get(
    connection_name='<Your Bing connection name>',  # e.g., "BingSearch"
)

conn_id = bing_connection.id

In [ ]:
async def web_ai_agent(query: str) -> str:
    # print("This is Bing for Azure AI Agent Service .......")
    bing = BingGroundingTool(connection_id=conn_id)
    with project_client:
        agent = project_client.agents.create_agent(
            model="gpt-4o",
            name="BingGroundingTool-Assistant",
            instructions="""        
                You are a web search agent.
                Your only tool is search_tool - use it to find information on the subject given.
                You will gather enough information on the subject that can create 500 words content.
                Please present the information in a structured format in multiple paragraphs.
            """,
            tools=bing.definitions,
            headers={"x-ms-enable-preview": "true"}
        )
        # print(f"Created agent, ID: {agent.id}")

        # Create thread for communication
        thread = project_client.agents.create_thread()
        # print(f"Created thread, ID: {thread.id}")

        # Create message to thread
        message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=query,
        )
        # print(f"SMS: {message}")
        # Create and process agent run in thread with tools
        run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
        print(f"Run finished with status: {run.status}")

        if run.status == "failed":
            print(f"Run failed: {run.last_error}")

        # Delete the assistant when done
        # project_client.agents.delete_agent(agent.id)
        # print("Deleted agent")

        # Fetch and log all messages
        messages = project_client.agents.list_messages(thread_id=thread.id)
        # print("Messages:"+ messages["data"][0]["content"][0]["text"]["value"])
    return messages["data"][0]["content"][0]["text"]["value"]



In [ ]:
bing_search_agent = AssistantAgent(
    name="assistant",
    model_client=az_model_client,
    tools=[web_ai_agent],
    system_message="Use tools to solve tasks."
)

In [ ]:
write_agent = AssistantAgent(
    name="write_agent",
    system_message="You are writer agent to write a blog based on the search result given. Keep the blog simple and easy to understand.",
    model_client=az_model_client
)

In [ ]:
# Define a termination condition that stops the task after 5 messages.
max_message_termination = MaxMessageTermination(5)
# Create a group chat with the agents
group_chat = RoundRobinGroupChat([bing_search_agent, write_agent],termination_condition=max_message_termination)

In [ ]:
await Console(
                group_chat.run_stream(task="""

                   I am writing a blog about Tesla. Find latest news for the following 3 questions and write a blog based on the search results
                    
                    1. How Tesla is performing?
                    2. Reliability of Tesla cars
                    3. Tesla's future plans                          

    """)
)  # Stream the messages to the console.